In [1]:
! pip install sentencepiece
! pip install fire
! git clone https://github.com/huggingface/transformers.git
! pip install transformers
! pip install datasets

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
     |████████████████████████████████| 87 kB 1.1 MB/s eta 0:00:011
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=59bc25c2152977ad5881d5338bd5d3134d3175e8b3619aea1c871a387c80de78
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Cloning into 'transformers'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 61230 (delta 7), reused 3 (delta 0), pack-reused 61206
Receiving objects: 100% (61230/61230), 46.02 MiB | 24.82 MiB/s, done.
Resolving deltas: 100% (43273/43273), done.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' co

In [2]:
! python transformers/examples/research_projects/seq2seq-distillation/make_student.py t5-base --save_path distil_t5_base_12_6 --e 12 --d 6

2021-02-03 07:59:10.486611: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2
Downloading: 100%|██████████████████████████| 1.20k/1.20k [00:00<00:00, 910kB/s]
Downloading: 100%|███████████████████████████| 792k/792k [00:00<00:00, 2.32MB/s]
Downloading: 100%|█████████████████████████| 1.39M/1.39M [00:00<00:00, 3.57MB/s]
Downloading: 100%|███████████████████████████| 892M/892M [00:19<00:00, 44.8MB/s]
Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a mod

In [3]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [6]:
from datasets import load_dataset
dataset = load_dataset("quora")

Using custom data configuration default


Dataset quora downloaded and prepared to /root/.cache/huggingface/datasets/quora/default/0.0.0/2be517cf0ac6de94b77a103a36b141347a13f40637fbebaccb56ddbe397876be. Subsequent calls will reuse this data.


In [8]:
sentences = []
for sample in dataset['train']:
  if sample['is_duplicate']:
    sentences.append({
        'sent_1': sample['questions']['text'][0],
        'sent_2': sample['questions']['text'][1],
    })

In [9]:
train_sentences = sentences[:18000]
valid_sentences = sentences[18000:20000]

In [10]:
class CustomDataset(Dataset):
  def __init__(self, tokenizer, text_list, max_input_length=64, max_target_length=64):
    self.tokenizer = tokenizer
    self.text_list = text_list
    self.max_input_length = max_input_length
    self.max_target_length = max_target_length
  
  def __len__(self):
    return len(self.text_list)
  
  def __getitem__(self, idx):
    sample = self.text_list[idx]
    sentence_1 = sample['sent_1']
    sentence_2 = sample['sent_2']
    tokens_1 = self.tokenizer(sentence_1, max_length=self.max_input_length, padding='max_length', truncation=True, return_tensors='pt')
    tokens_2 = self.tokenizer(sentence_2, max_length=self.max_target_length, padding='max_length', truncation=True, return_tensors='pt')
    return {
        'inp_ids': tokens_1['input_ids'],
        'tgt_ids': tokens_2['input_ids'],
    }

In [11]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained('distil_t5_base_12_6')
model = T5ForConditionalGeneration.from_pretrained('distil_t5_base_12_6')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [12]:
params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Total Trainable Parameters: ', params)

Total Trainable Parameters:  166266624


In [13]:
BATCH_SIZE = 32
LEARNING_RATE = 3e-5
EPOCHS = 2

In [14]:
train_dataset = CustomDataset(tokenizer, train_sentences, max_input_length=85, max_target_length=85)
valid_dataset = CustomDataset(tokenizer, valid_sentences, max_input_length=85, max_target_length=85)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

print('Length of Train Loader: ', len(train_loader))
print('Length of Valid Loader: ', len(valid_loader))

Length of Train Loader:  563
Length of Valid Loader:  63


In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [16]:
def compute_loss(model, data_loader, device):
  list_loss = []
  for sample in data_loader:
    inp_ids = sample['inp_ids'].squeeze(1).to(device)
    tgt_ids = sample['tgt_ids'].squeeze(1).to(device)
    loss = model(input_ids=inp_ids, labels=tgt_ids).loss
    list_loss.append(loss.item())
  return torch.mean(torch.tensor(list_loss))

start_time = time.time()
for epoch in range(EPOCHS):
  model.train()
  for idx, sample in enumerate(train_loader):
    inp_ids = sample['inp_ids'].squeeze(1).to(device)
    tgt_ids = sample['tgt_ids'].squeeze(1).to(device)
    loss = model(input_ids=inp_ids, labels=tgt_ids).loss

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Logging
    if idx % 200 == 0:
      print('Epoch: %04d/%04d || Batch: %04d/%04d || Loss: %.2f' % (epoch+1, EPOCHS, idx, len(train_loader), loss.item()))
  
  model.eval()
  with torch.set_grad_enabled(False):
    valid_loss = compute_loss(model, valid_loader, device)
    print('Valid Loss: %.2f' % (valid_loss))
  epoch_elapsed_time = (time.time() - start_time) / 60
  print('Epoch Elapsed Time: %.2f' % (epoch_elapsed_time))
total_training_time = (time.time() - start_time) / 60
print('Total Training Time: %.2f' % (total_training_time))

Epoch: 0001/0003 || Batch: 0000/0563 || Loss: 20.70
Epoch: 0001/0003 || Batch: 0200/0563 || Loss: 0.51
Epoch: 0001/0003 || Batch: 0400/0563 || Loss: 0.39
Valid Loss: 0.30
Epoch Elapsed Time: 5.52
Epoch: 0002/0003 || Batch: 0000/0563 || Loss: 0.30
Epoch: 0002/0003 || Batch: 0200/0563 || Loss: 0.38
Epoch: 0002/0003 || Batch: 0400/0563 || Loss: 0.24
Valid Loss: 0.27
Epoch Elapsed Time: 11.02
Epoch: 0003/0003 || Batch: 0000/0563 || Loss: 0.33
Epoch: 0003/0003 || Batch: 0200/0563 || Loss: 0.26
Epoch: 0003/0003 || Batch: 0400/0563 || Loss: 0.30
Valid Loss: 0.26
Epoch Elapsed Time: 16.53
Total Training Time: 16.53


In [19]:
start_time = time.time()
text = "Can I really become financial independent if I spend money wisely?"
return_seq = 3
model.eval()
with torch.set_grad_enabled(False):
  tokens = tokenizer(text, max_length=85, truncation=True, padding=True, return_tensors='pt')
  ids = tokens['input_ids'].to(device)
  mask = tokens['attention_mask'].to(device)
  outputs = model.generate(input_ids=ids, num_beams=return_seq * 5, early_stopping=True, max_length=85, num_return_sequences=return_seq, top_k=50, top_p=0.98)
  for i in range(return_seq):
    print(tokenizer.decode(outputs[i], skip_special_tokens=True))
end_time = (time.time() - start_time)
print('Elapsed Time: %.2f sec' % (end_time))

how can I become financially independent?
how do I become financially independent?
can I become financially independent?
Elapsed Time: 0.18 sec
